# A* Algorythm

## Pramble

[Basically explain here what it is]

## Short history

[talk about the shakey robot, why this algorithm was needed, and what it is used for today]

## Core
[This is the meat a good chunk of the effort will go into here. Talk about the structure, strenghts against other methods, weaknesses. Talk about big O notation and how it relates to this]

## Demo
[Try to visualize the algorithm]

## Variants
[There are many parameters and variations. Pick a few to dive into about differences and why and where they may be used and briefly talk about others]

## Conclutions
[Summarize everything here]